## Module for building Henderson's Mixed Model Equations for single trait including marker  and polygenic effect



### <font color="red"> Data</font>

In [1]:
include("../src/JWAS.jl")

JWAS

In [2]:
using DataFrames
using QTLDatasets
using JWAS.MMEModule

In [3]:
phenofile = QTLDatasets.dataset("testMME","simple.txt")
genofile = QTLDatasets.dataset("testMME","genotype.txt")
pedfile   = QTLDatasets.dataset("testMME","pedigree.txt")

"/Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMME/pedigree.txt"

In [4]:
phenotype = readtable(phenofile,separator = ',',header=true)

,Animal,Age,y
1,S1,1,-0.92
2,D1,2,-1.05
3,O1,3,-0.92
4,O3,2,1.2


In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1,0,1,1,1
D1,2,0,2,2,1
O1,1,2,0,1,0
O3,0,0,2,1,1


### <font color="red"> Run module</font>

In [6]:
varRes = 1.0
varGen = 2.5
markerProp = 0.8
MVarGen= varGen*markerProp
PVarGen= varGen*(1-markerProp);

In [7]:
ped = get_pedigree(pedfile);

In [8]:
varRes=1.0
mme = build_model("y = intercept + Age + Animal",varRes)
set_covariate(mme,"Age")
set_random(mme,"Animal",ped,PVarGen)

### <font color="red"> work with markers</font>

In [9]:
add_markers(mme,genofile,MVarGen,separator=',');

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMME/genotype.txt is ,  .


### <font color="red"> run MCMC</font>

In [10]:
output=runMCMC(mme,phenotype,chain_length=50000,
                         printout_frequency=5000,methods="BayesC",Pi=0.8,
                         output_marker_effects_frequency=1000)

at sample: 5000 with meanVare: 1.1255339248360174
at sample: 10000 with meanVare: 1.157863608313913
at sample: 15000 with meanVare: 1.1518654318533825
at sample: 20000 with meanVare: 1.1532500036292794
at sample: 25000 with meanVare: 1.148365388495088
at sample: 30000 with meanVare: 1.1605766679042304
at sample: 35000 with meanVare: 1.1509132261386548
at sample: 40000 with meanVare: 1.1497864322883686
at sample: 45000 with meanVare: 1.1587688281537294
at sample: 50000 with meanVare: 1.1633085620213275


Dict{Any,Any} with 4 entries:
  "Posterior Mean of Loca… => 7x2 Array{Any,2}:…
  "MCMC samples for resid… => [0.657129,0.194248,0.454677,0.558346,0.504454,0.5…
  "MCMC samples for genet… => 50000x1 Array{Float64,2}:…
  "Posterior Mean of Mark… => [-0.2829441564991788,-0.1409481105736366,0.090936…

In [11]:
keys(output)

Base.KeyIterator for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior Mean of Location Parameters"
  "MCMC samples for residual variance"
  "MCMC samples for genetic var-cov parameters"
  "Posterior Mean of Marker Effects"

In [12]:
output["Posterior Mean of Location Parameters"]

7x2 Array{Any,2}:
 "intercept: intercept"  -0.985575  
 "Age: Age"               0.276339  
 "Animal: S1"             0.00398979
 "Animal: D1"            -0.0146501 
 "Animal: O1"            -0.0370037 
 "Animal: O3"             0.0923229 
 "Animal: O2"            -0.0065065 

In [13]:
output["Posterior Mean of Marker Effects"]

5-element Array{Float64,1}:
 -0.282944 
 -0.140948 
  0.0909368
 -0.111449 
  0.139766 

In [14]:
output[ "MCMC samples for genetic var-cov parameters"]

50000x1 Array{Float64,2}:
 0.270213 
 0.450931 
 0.31171  
 0.161513 
 0.273327 
 0.257452 
 0.0955835
 0.198121 
 0.281785 
 0.146857 
 0.231007 
 0.222213 
 0.15239  
 ⋮        
 0.283917 
 0.353707 
 0.142414 
 0.48211  
 0.205283 
 0.383252 
 0.284798 
 0.324698 
 0.273556 
 0.28646  
 0.113053 
 0.168873 

In [15]:
output["MCMC samples for residual variance"]

50000-element Array{Float64,1}:
 0.657129
 0.194248
 0.454677
 0.558346
 0.504454
 0.516481
 0.460213
 0.833731
 0.759667
 1.04109 
 1.7301  
 1.19605 
 1.35431 
 ⋮       
 0.64421 
 0.524068
 0.670135
 1.35558 
 0.880981
 0.851161
 1.99047 
 0.933943
 0.431982
 0.331644
 0.281757
 0.257994